### Loading and downloading dependencies

In [9]:
!pip install -U "transformers<5" "huggingface-hub<1" tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 21.5 MB/s eta 0:00:00
    torch (>=1.8.*)
           ~~~~~~^
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.4.1
    Uninstalling huggingface_hub-1.4.1:━━━━━━━━━ 0/2 [huggingface-hub]
      Successfully uninstalled huggingface_hub-1.4.1 0/2 [huggingface-hub]
  Attempting uninstall: transformers━━━━━━━━━━━━ 0/2 [huggingface-hub]
    Found existing installation: transformers 5.1.032m0/2 [huggingface-hub]
    Uninstalling transformers-5.1.0:m╺━━━━━━━━━━━━━━━━━━━ 1/2 [transformers]
      Successfully uninstalled transformers-5.1.0━━━━━━━━━━━━━━━━━ 1/2 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [2]:
!pip install sentence_transformers

    torch (>=1.8.*)
           ~~~~~~^


In [10]:
from __future__ import annotations
import ast
import re
from dataclasses import dataclass
from typing import List, Dict, Tuple
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer


In [12]:
# ----------------------------
# Config
# ----------------------------
TARGET_LABELS = [
    "melanoma",
    "psoriasis",
    "systemic lupus erythematosus",
    "vitiligo",
    "dermatitis",
    "scleroderma",
]
UNCLEAR = "unclear"

LABEL_PROTOTYPES = {
    "melanoma": [
        "melanoma malignant melanoma cutaneous melanoma skin cancer melanocytic tumor",
        "melanoma in situ metastatic melanoma",
    ],
    "psoriasis": [
        "psoriasis chronic plaque psoriasis guttate psoriasis pustular psoriasis",
        "psoriatic disease erythematous scaly plaques",
    ],
    "systemic lupus erythematosus": [
        "systemic lupus erythematosus SLE autoimmune connective tissue disease",
        "cutaneous lupus lupus erythematosus photosensitive rash",
    ],
    "vitiligo": [
        "vitiligo acquired depigmentation autoimmune loss of melanocytes",
        "depigmented macules and patches vitiligo",
    ],
    "dermatitis": [
        "dermatitis eczema atopic dermatitis spongiotic dermatitis",
        "contact dermatitis allergic contact dermatitis irritant contact dermatitis",
    ],
    "scleroderma": [
        "scleroderma systemic sclerosis localized scleroderma morphea",
        "skin thickening induration scleroderma",
    ],
}

MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
# or smaller/faster:
# MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)
MIN_SCORE = 0.45
MIN_MARGIN = 0.06

DROP_IF_CONTAINS = [
    "remove from labeling tool",
    "o/e -",
]

# ----------------------------
# Helpers
# ----------------------------
def parse_conditions(raw) -> list[str]:
    """raw is usually a string like "['Eczema','Psoriasis']" or NaN."""
    if pd.isna(raw):
        return []
    s = str(raw).strip()
    if s in ("", "[]"):
        return []
    try:
        items = ast.literal_eval(s)
        if not isinstance(items, list):
            return []
    except Exception:
        return []

    out = []
    for x in items:
        if not isinstance(x, str):
            continue
        t = re.sub(r"\s+", " ", x.strip().lower())
        if not t:
            continue
        if any(p in t for p in DROP_IF_CONTAINS):
            continue
        out.append(t)
    return out


def assign_label(conditions: list[str], model, proto_emb, label_to_proto_idx) -> tuple[str, dict]:
    if not conditions:
        return UNCLEAR, {l: 0.0 for l in TARGET_LABELS}

    cond_emb = model.encode(conditions, normalize_embeddings=True)
    sim = cond_emb @ proto_emb.T  # cosine sim because embeddings normalized

    scores = {}
    for lab in TARGET_LABELS:
        idx = label_to_proto_idx[lab]
        scores[lab] = float(sim[:, idx].max())

    ranked = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
    best_lab, best_score = ranked[0]
    second_score = ranked[1][1] if len(ranked) > 1 else -1.0

    if best_score < MIN_SCORE or (best_score - second_score) < MIN_MARGIN:
        return UNCLEAR, scores
    return best_lab, scores


# ----------------------------
# Load dataframe
# ----------------------------
df2 = pd.read_csv("/home/provido/provido/luna/image_data/scin_meta/dataset_scin_labels.csv")

# This column should contain strings that look like lists: "['Eczema', ...]"
col = "dermatologist_skin_condition_on_label_name"

# Parse into Python lists
df2["conditions_list"] = df2[col].apply(parse_conditions)

# ----------------------------
# Build model + prototypes
# ----------------------------
model = SentenceTransformer(MODEL_NAME)

proto_texts = []
proto_owner = []
for lab in TARGET_LABELS:
    for desc in LABEL_PROTOTYPES[lab]:
        proto_texts.append(desc.lower())
        proto_owner.append(lab)

proto_emb = model.encode(proto_texts, normalize_embeddings=True)

label_to_proto_idx = {l: [] for l in TARGET_LABELS}
for i, lab in enumerate(proto_owner):
    label_to_proto_idx[lab].append(i)

# ----------------------------
# Assign labels
# ----------------------------
assigned = []
scores_list = []

for conds in df2["conditions_list"].tolist():
    lab, scores = assign_label(conds, model, proto_emb, label_to_proto_idx)
    assigned.append(lab)
    scores_list.append(scores)

df2["assigned_target_label"] = assigned
df2["assigned_target_scores"] = scores_list  # dict per row (optional but useful)

# Example: see distribution
print(df2["assigned_target_label"].value_counts(dropna=False))

# Save if you want
# df2.to_csv("dataset_scin_labels_with_targets.csv", index=False)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

assigned_target_label
unclear                         2626
dermatitis                      2015
psoriasis                        264
systemic lupus erythematosus      49
vitiligo                          48
melanoma                          21
scleroderma                       10
Name: count, dtype: int64


In [ ]:
# Filter out empty lists from the result
filtered_list = [x for x in df2[df2['assigned_target_label'] == "unclear"]['dermatologist_skin_condition_on_label_name'].tolist() if x != '[]']
filtered_list

["['Prurigo nodularis', 'Actinic Keratosis', 'SCC/SCCIS']",
 "['Urticaria', 'Leukocytoclastic Vasculitis', 'Urticaria', 'Insect Bite', 'Leukocytoclastic Vasculitis']",
 "['Inflicted skin lesions', 'Basal Cell Carcinoma', 'SCC/SCCIS']",
 "['Purpura']",
 "['Lichen planus/lichenoid eruption', 'Molluscum Contagiosum']",
 "['Onychomycosis', 'Onychodystrophy']",
 "['Chronic dermatitis, NOS']",
 "['Tinea']",
 "['Drug Rash', 'Leukocytoclastic Vasculitis']",
 "['Granuloma annulare']",
 "['Verruca vulgaris', 'SK/ISK']",
 "['Granuloma annulare', 'Annular erythema']",
 "['Condyloma acuminatum', 'Molluscum Contagiosum']",
 "['Contact purpura', 'Purpura', 'Vasculitis of the skin, NOS']",
 "['Miliaria', 'Acute generalised exanthematous pustulosis']",
 "['Impetigo', 'Granuloma faciale', 'Cutaneous sarcoidosis']",
 '["Kaposi\'s sarcoma of skin", \'O/E - ecchymoses present\', \'Leukocytoclastic Vasculitis\']',
 "['Verruca vulgaris']",
 "['Impetigo']",
 "['Erosion of skin', 'Abrasion, scrape, or scab', '

### Less conservative mapping

In [3]:
import ast
import re
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# ----------------------------
# Targets
# ----------------------------
TARGET_LABELS = [
    "melanoma",
    "psoriasis",
    "systemic lupus erythematosus",
    "vitiligo",
    "dermatitis",
    "scleroderma",
]
UNCLEAR = "unclear"

EXACT_ALIASES = {
    "melanoma": {"melanoma", "malignant melanoma", "cutaneous melanoma"},
    "psoriasis": {"psoriasis", "plaque psoriasis", "guttate psoriasis", "pustular psoriasis"},
    "systemic lupus erythematosus": {
        "systemic lupus erythematosus",
        "sle",
        "sle - systemic lupus erythematosus-related syndrome",
        "cutaneous lupus",
        "lupus erythematosus",
    },
    "vitiligo": {"vitiligo"},
    "dermatitis": {
        "dermatitis",
        "eczema",
        "atopic dermatitis",
        "allergic contact dermatitis",
        "irritant contact dermatitis",
        "contact dermatitis, nos",
        "cd - contact dermatitis",
        "acute dermatitis, nos",
        "chronic dermatitis, nos",
        "lichenified eczematous dermatitis",
        "stasis dermatitis",
        "dermatosis",
    },
    "scleroderma": {"scleroderma", "systemic sclerosis", "morphea", "localized scleroderma"},
}

LABEL_PROTOTYPES = {
    "melanoma": ["melanoma malignant melanoma cutaneous melanoma skin cancer melanocytic tumor"],
    "psoriasis": ["psoriasis chronic plaque psoriasis guttate psoriasis pustular psoriasis scaly plaques"],
    "systemic lupus erythematosus": ["systemic lupus erythematosus SLE cutaneous lupus lupus erythematosus autoimmune"],
    "vitiligo": ["vitiligo depigmentation loss of pigment melanocytes"],
    "dermatitis": ["dermatitis eczema atopic dermatitis contact dermatitis allergic irritant spongiotic dermatitis"],
    "scleroderma": ["scleroderma systemic sclerosis morphea skin thickening induration"],
}

MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
MIN_SCORE = 0.42
MIN_MARGIN = 0.04

DROP_IF_CONTAINS = ["remove from labeling tool", "o/e -"]

def parse_conditions(raw) -> list[str]:
    if pd.isna(raw):
        return []
    s = str(raw).strip()
    if s in ("", "[]"):
        return []
    try:
        items = ast.literal_eval(s)
        if not isinstance(items, list):
            return []
    except Exception:
        return []

    out = []
    for x in items:
        if not isinstance(x, str):
            continue
        t = re.sub(r"\s+", " ", x.strip().lower())
        if not t:
            continue
        if any(p in t for p in DROP_IF_CONTAINS):
            continue
        out.append(t)
    return out

def build_prototypes(model):
    proto_texts, proto_owner = [], []
    for lab in TARGET_LABELS:
        for desc in LABEL_PROTOTYPES[lab]:
            proto_texts.append(desc.lower())
            proto_owner.append(lab)
    proto_emb = model.encode(proto_texts, normalize_embeddings=True)
    label_to_idx = {l: [] for l in TARGET_LABELS}
    for i, lab in enumerate(proto_owner):
        label_to_idx[lab].append(i)
    return proto_emb, label_to_idx

def first_entry_target_or_none(conditions: list[str]) -> str | None:
    """Return target label if the FIRST condition is an exact alias match; else None."""
    if not conditions:
        return None
    first = conditions[0]
    for lab, aliases in EXACT_ALIASES.items():
        if first in aliases:
            return lab
    return None

def all_target_hits_in_order(conditions: list[str]) -> list[str]:
    """Return all target labels that appear (via exact alias) in the order they first appear."""
    hits = []
    for c in conditions:
        for lab, aliases in EXACT_ALIASES.items():
            if c in aliases and lab not in hits:
                hits.append(lab)
    return hits

def semantic_pick_excluding(conditions, model, proto_emb, label_to_proto_idx, exclude: set[str]):
    """Semantic pick among TARGET_LABELS minus exclude set, with same thresholds."""
    cond_emb = model.encode(conditions, normalize_embeddings=True)
    sim = cond_emb @ proto_emb.T

    scores = {}
    for lab in TARGET_LABELS:
        if lab in exclude:
            continue
        idx = label_to_proto_idx[lab]
        scores[lab] = float(sim[:, idx].max())

    ranked = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
    best_lab, best_score = ranked[0]
    second_score = ranked[1][1] if len(ranked) > 1 else -1.0

    if best_score < MIN_SCORE or (best_score - second_score) < MIN_MARGIN:
        return UNCLEAR, scores
    return best_lab, scores

def assign_label(conditions: list[str], model, proto_emb, label_to_proto_idx):
    if not conditions:
        return UNCLEAR, {l: 0.0 for l in TARGET_LABELS}

    hits_in_order = all_target_hits_in_order(conditions)

    # If at least one target appears:
    if hits_in_order:
        # Pick by FIRST entry (i.e., target corresponding to earliest condition)
        first_hit = hits_in_order[0]

        # If first hit is dermatitis, pick the next closest target label instead
        if first_hit == "dermatitis":
            lab, scores = semantic_pick_excluding(
                conditions, model, proto_emb, label_to_proto_idx, exclude={"dermatitis"}
            )
            return lab, scores

        # Otherwise choose the first-hit target deterministically
        return first_hit, {l: (1.0 if l == first_hit else 0.0) for l in TARGET_LABELS}

    # No explicit target present -> semantic fallback across all targets
    return semantic_pick_excluding(conditions, model, proto_emb, label_to_proto_idx, exclude=set())

# ----------------------------
# Run on df2
# ----------------------------
df2 = pd.read_csv("/home/provido/provido/luna/image_data/scin_meta/dataset_scin_labels.csv")
col = "dermatologist_skin_condition_on_label_name"

df2["conditions_list"] = df2[col].apply(parse_conditions)

model = SentenceTransformer(MODEL_NAME)
proto_emb, label_to_proto_idx = build_prototypes(model)

assigned, scores_list = [], []
for conds in df2["conditions_list"].tolist():
    lab, scores = assign_label(conds, model, proto_emb, label_to_proto_idx)
    assigned.append(lab)
    scores_list.append(scores)

df2["assigned_target_label"] = assigned
df2["assigned_target_scores"] = scores_list

print(df2["assigned_target_label"].value_counts(dropna=False))

assigned_target_label
unclear                         3964
psoriasis                        493
dermatitis                       353
systemic lupus erythematosus     122
melanoma                          56
scleroderma                       26
vitiligo                          19
Name: count, dtype: int64


In [13]:
df2.to_csv("/home/provido/provido/luna/image_data/scin_meta/scin_assigned_target_labels.csv", index = False)

In [ ]:
We proceed with this target list script as dermatitis is bound to be present when other conditions are there. 
Derm is likely to be present when psoriasis and SLE but not vice versa.
assigned_target_label
unclear                         3964
psoriasis                        493
dermatitis                       353
systemic lupus erythematosus     122
melanoma                          56
scleroderma                       26
vitiligo                          19
Name: count, dtype: int64

In [10]:
df2.columns

Index(['case_id', 'dermatologist_gradable_for_skin_condition_1',
       'dermatologist_gradable_for_skin_condition_2',
       'dermatologist_gradable_for_skin_condition_3',
       'dermatologist_skin_condition_on_label_name',
       'dermatologist_skin_condition_confidence',
       'weighted_skin_condition_label',
       'dermatologist_gradable_for_fitzpatrick_skin_type_1',
       'dermatologist_gradable_for_fitzpatrick_skin_type_2',
       'dermatologist_gradable_for_fitzpatrick_skin_type_3',
       'dermatologist_fitzpatrick_skin_type_label_1',
       'dermatologist_fitzpatrick_skin_type_label_2',
       'dermatologist_fitzpatrick_skin_type_label_3',
       'gradable_for_monk_skin_tone_india', 'gradable_for_monk_skin_tone_us',
       'monk_skin_tone_label_india', 'monk_skin_tone_label_us',
       'conditions_list', 'assigned_target_label', 'assigned_target_scores'],
      dtype='object')

In [21]:
df = pd.read_csv("/home/provido/provido/luna/image_data/scin_meta/scin_cases.csv")
df.columns

Index(['case_id', 'source', 'release', 'year', 'age_group', 'sex_at_birth',
       'fitzpatrick_skin_type',
       'race_ethnicity_american_indian_or_alaska_native',
       'race_ethnicity_asian', 'race_ethnicity_black_or_african_american',
       'race_ethnicity_hispanic_latino_or_spanish_origin',
       'race_ethnicity_middle_eastern_or_north_african',
       'race_ethnicity_native_hawaiian_or_pacific_islander',
       'race_ethnicity_white', 'race_ethnicity_other_race',
       'race_ethnicity_prefer_not_to_answer', 'textures_raised_or_bumpy',
       'textures_flat', 'textures_rough_or_flaky', 'textures_fluid_filled',
       'body_parts_head_or_neck', 'body_parts_arm', 'body_parts_palm',
       'body_parts_back_of_hand', 'body_parts_torso_front',
       'body_parts_torso_back', 'body_parts_genitalia_or_groin',
       'body_parts_buttocks', 'body_parts_leg', 'body_parts_foot_top_or_side',
       'body_parts_foot_sole', 'body_parts_other',
       'condition_symptoms_bothersome_appearan

In [ ]:
df[df['related_category'] == 'LOOKS_HEALTHY']

,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,related_category,condition_duration,image_1_path,image_2_path,image_3_path,image_1_shot_type,image_2_shot_type,image_3_shot_type,combined_race,race_ethnicity_two_or_more_after_mitigation
13,-1036981968706453829,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,NaN,NaN,NaN,...,LOOKS_HEALTHY,NaN,dataset/images/2919857728265265465.png,NaN,NaN,CLOSE_UP,NaN,NaN,NaN,NaN
14,-1037438006618214143,SCIN,1.0.0,2023,AGE_18_TO_29,FEMALE,FST6,NaN,NaN,YES,...,LOOKS_HEALTHY,ONE_DAY,dataset/images/-7305545407741188289.png,NaN,NaN,CLOSE_UP,NaN,NaN,BLACK_OR_AFRICAN_AMERICAN,NaN
22,-1067542267811956200,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,NaN,NaN,NaN,...,LOOKS_HEALTHY,NaN,dataset/images/-2682298177796377023.png,dataset/images/6195645539054445183.png,NaN,AT_AN_ANGLE,CLOSE_UP,NaN,NaN,NaN
26,-1079727161055493298,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,NaN,NaN,NaN,...,LOOKS_HEALTHY,LESS_THAN_ONE_WEEK,dataset/images/1303931515034925972.png,dataset/images/17775584599697809.png,dataset/images/691263588284095124.png,AT_DISTANCE,AT_AN_ANGLE,CLOSE_UP,NaN,NaN
33,-111683086443147412,SCIN,1.0.0,2023,AGE_UNKNOWN,MALE,FST3,NaN,NaN,NaN,...,LOOKS_HEALTHY,ONE_TO_FOUR_WEEKS,dataset/images/-304171554496873031.png,dataset/images/1628199398892577507.png,NaN,CLOSE_UP,AT_AN_ANGLE,NaN,HISPANIC_LATINO_OR_SPANISH_ORIGIN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000,9176882468663374673,SCIN,1.0.0,2023,AGE_60_TO_69,MALE,FST4,NaN,NaN,NaN,...,LOOKS_HEALTHY,LESS_THAN_ONE_WEEK,dataset/images/-2522232535157376613.png,dataset/images/6846875640294463690.png,dataset/images/7164680788602110344.png,AT_AN_ANGLE,CLOSE_UP,AT_DISTANCE,WHITE,NaN
5012,9212720578959463880,SCIN,1.0.0,2023,AGE_40_TO_49,FEMALE,NONE_IDENTIFIED,NaN,NaN,NaN,...,LOOKS_HEALTHY,ONE_TO_FOUR_WEEKS,dataset/images/6207991513722307450.png,NaN,NaN,CLOSE_UP,NaN,NaN,WHITE,NaN
5019,941421903708181943,SCIN,1.0.0,2023,AGE_30_TO_39,FEMALE,NaN,NaN,NaN,NaN,...,LOOKS_HEALTHY,ONE_DAY,dataset/images/-1892382246747642071.png,dataset/images/-8336982412057653126.png,dataset/images/152869571406128721.png,AT_AN_ANGLE,CLOSE_UP,AT_DISTANCE,WHITE,NaN
5025,963694477603925530,SCIN,1.0.0,2023,AGE_50_TO_59,MALE,FST4,NaN,NaN,NaN,...,LOOKS_HEALTHY,ONE_DAY,dataset/images/-1642667040160094612.png,NaN,NaN,CLOSE_UP,NaN,NaN,WHITE,NaN


In [17]:
df2[df2['dermatologist_skin_condition_on_label_name'] == '[]']

,case_id,dermatologist_gradable_for_skin_condition_1,dermatologist_gradable_for_skin_condition_2,dermatologist_gradable_for_skin_condition_3,dermatologist_skin_condition_on_label_name,dermatologist_skin_condition_confidence,weighted_skin_condition_label,dermatologist_gradable_for_fitzpatrick_skin_type_1,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us,conditions_list,assigned_target_label,assigned_target_scores
3,-1003826561155964328,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},NO,NaN,NaN,NaN,NaN,NaN,True,True,2.0,4.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
12,-1036016816671158395,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},YES,NaN,NaN,FST3,NaN,NaN,True,True,3.0,2.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
14,-1037438006618214143,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},YES,NaN,NaN,FST4,NaN,NaN,True,True,4.0,3.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
16,-1056272893472010635,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},NO,NaN,NaN,NaN,NaN,NaN,True,True,2.0,3.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
20,-1061801124542504778,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},YES,NaN,NaN,FST3,NaN,NaN,True,True,2.0,2.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5020,943125820577516833,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,2.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
5022,953667114257627038,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},YES,NaN,NaN,FST3,NaN,NaN,True,True,2.0,2.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
5023,954707330462098233,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},NO,NaN,NaN,NaN,NaN,NaN,True,True,4.0,2.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
5026,964881286184961934,NO_IMAGE_QUALITY_INSUFFICIENT,NaN,NaN,[],[],{},YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,3.0,[],unclear,"{'melanoma': 0.0, 'psoriasis': 0.0, 'systemic ..."
